In [39]:
import PIL
import numpy as np 
import os
import re
from PIL import Image
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import keras
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense

#import re
#from keras.utils import to_categorical

In [48]:
print(keras.optimizers.Adam(lr=0.01))


In [59]:

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size =(4,4), activation='relu',input_shape = input_shape))##(113x113x16)
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))## (81x81x32)
batch_norm()
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=(2,2), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=128, kernel_size=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=256, kernel_size=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='softmax'))

model.summary()
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer_fn = keras.optimizers.Adam(lr=0.01, decay=1e-6)
model.compile(optimizer=optimizer_fn, loss=loss_fn, metrics=['accuracy'])    

ValueError: Input 0 is incompatible with layer conv2d_94: expected ndim=4, found ndim=5

In [26]:
train_path = r"C:\Users\Abhishek\Desktop\keras_try\train.csv"
test_path = r"C:\Users\Abhishek\Desktop\keras_try\test.csv"
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
print(train_df.head())

                 id                                                url
0  cb9998b8cdaf6235  https://lh3.googleusercontent.com/-q8B91vDIQZY...
1  30728cf6e50a6bc6  https://lh3.googleusercontent.com/-91gJSKTgv5Q...
2  16afbc86b710337d  https://lh3.googleusercontent.com/-GHZdXuf2wMg...
3  d29b2166cf522450  https://lh3.googleusercontent.com/-cWDnYNQhyws...
4  dd5c03b20c21cfba  https://lh3.googleusercontent.com/-PSLN6BloM-k...


In [27]:
sampling_rate = 0.02
random_state = 0
landmarks=train_df.groupby(by='landmark_id').count().loc[:,"id"]
lmark_descend = landmarks.sort_values(ascending=False)[0:101]
#print(lmark_descend.index)  

train_ordered = train_df[train_df.landmark_id.isin(lmark_descend.index)]
#print(train_ordered)
sample_gby = train_ordered.groupby(by = "landmark_id").apply(lambda x: x.sample(frac=sampling_rate, random_state=random_state))

sample_idx = sample_gby.index.levels[1]
train_sample = train_df.iloc[sample_idx, :]
print(train_sample)

                       id                                                url  \
782      8eb0ef107f9d7d84  http://lh6.ggpht.com/-Ea-useaeDrg/R-na8eB_8rI/...   
960      fbc10425aad4cb54  http://lh4.ggpht.com/-2MZrXMPSLw0/RnUp9VYNYYI/...   
1287     866ae5aeb8ac8ff9  https://lh4.googleusercontent.com/-Toq7n7DLZPk...   
1830     87718c752b782e3f  http://lh5.ggpht.com/-2vNIqWW_LTc/SSn01Rw6TxI/...   
1904     8d6ef2913366c8f1  http://lh3.ggpht.com/-m7xtm_ZI4So/Tiv5Cp-l2SI/...   
2009     95bdda5ca38eaa91  https://lh3.googleusercontent.com/-pnAwBOVTGnU...   
2281     4be9afadac49ddbe  https://lh5.googleusercontent.com/-k6uK-PGbc0w...   
2389     b159c578ddbd83cd  https://lh3.googleusercontent.com/-yi9kf8HdaDo...   
2634     c15ab5dc5ea52764  http://lh4.ggpht.com/-11CefaFo_6A/S8y7QnOl0NI/...   
2730     b37300b72d53e643  https://lh6.googleusercontent.com/-Rb-m3IfMdyw...   
2879     ef1258022b39f976  https://lh6.googleusercontent.com/-AxvO240eyQo...   
2945     dea49188ba49c70b  https://lh6.g

In [15]:
re_sampled_train_path = r"C:\Users\Abhishek\Desktop\keras_try\re-trained_data.csv"
resampled_pd = pd.read_csv(re_sampled_train_path)


In [21]:
def convert_img_to_array(path,sample):
    img_label_dict = {}
    sample_match = path.split('\\')
    counter =0
    if "Train_images" in sample_match:
        type_of_dataset = "Train_images"
    else:
        type_of_dataset = "Validation_images"
    
    all_files = os.listdir(path)
    full_file_path = [path+"\\"+each_file for each_file in all_files ]
    full_file_path =[each_file.replace('\\','/') for each_file in full_file_path]
    each_file_path, labels_out = [],[]
    for each_file in full_file_path:
        match = re.search("(?<={0}/)\S+(?=.jpg)".format(type_of_dataset),each_file)
        uniq_id = sample[sample["id"]==match.group()]['landmark_id']
        #print(uniq_id.item())
        #print(each_file)
        
        try:
            labels_out.append(uniq_id.item())
        except ValueError:
            continue
        each_file_path.append(each_file)
    
    processed_df = pd.DataFrame({"full file path":each_file_path,"label":labels_out})
        #img_label_dict[] = uniq_id
        
    return processed_df    

In [22]:
path = r"C:\Users\Abhishek\Desktop\keras_try\Train_images"
img_path_class_df = convert_img_to_array(path,resampled_pd)
#print(img_path_class_df['full file path'])


In [18]:
train_gen = ImageDataGenerator(rescale=1/255,)

In [19]:
train_generator = train_gen.flow_from_dataframe(
    dataframe = img_path_class_df,
    directory=None,
    x_col="full file path",
    y_col="label",
    weight_col=None,
    target_size=(128,128),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None)

Found 7407 images belonging to 100 classes.


In [52]:
epochs = 2
checkpointer = ModelCheckpoint(filepath=r'C:\Users\Abhishek\Desktop\keras_try\saved_models\weights.best.from_baseCNN.hdf5', 
                               verbose=1, save_best_only=True)

hist = model.fit_generator(train_generator,epochs=10,steps_per_epoch=32)
    

Epoch 1/10
32/32 [==============================] - 60s 2s/step - loss: 0.0544 - acc: 0.1006
Epoch 2/10
32/32 [==============================] - 46s 1s/step - loss: 0.0494 - acc: 0.1213
Epoch 3/10
32/32 [==============================] - 33s 1s/step - loss: 0.0497 - acc: 0.1240
Epoch 4/10
32/32 [==============================] - 33s 1s/step - loss: 0.0485 - acc: 0.1211
Epoch 5/10
32/32 [==============================] - 35s 1s/step - loss: 0.0493 - acc: 0.1035
Epoch 6/10
32/32 [==============================] - 33s 1s/step - loss: 0.0491 - acc: 0.1191
Epoch 7/10
32/32 [==============================] - 33s 1s/step - loss: 0.0491 - acc: 0.1318
Epoch 8/10
32/32 [==============================] - 33s 1s/step - loss: 0.0476 - acc: 0.1338
Epoch 9/10
32/32 [==============================] - 33s 1s/step - loss: 0.0490 - acc: 0.1221
Epoch 10/10
32/32 [==============================] - 34s 1s/step - loss: 0.0484 - acc: 0.1270


In [29]:

convert_img_to_array(path)

000edceb2acb414b.jpg
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x1200 at 0xA7672F3358>
[[[151 182 213]
  [151 182 213]
  [151 182 213]
  ...
  [211 218 224]
  [212 219 225]
  [212 219 225]]

 [[151 182 213]
  [151 182 213]
  [151 182 213]
  ...
  [212 219 225]
  [212 219 225]
  [213 220 226]]

 [[151 182 213]
  [151 182 213]
  [151 182 213]
  ...
  [213 220 226]
  [213 220 226]
  [213 220 226]]

 ...

 [[130 131 126]
  [130 131 126]
  [131 132 127]
  ...
  [116 119 110]
  [116 119 110]
  [116 119 110]]

 [[119 118 114]
  [122 121 117]
  [126 125 121]
  ...
  [119 122 113]
  [119 122 113]
  [119 122 113]]

 [[ 80  79  75]
  [ 84  83  79]
  [ 90  89  85]
  ...
  [122 125 116]
  [122 125 116]
  [122 125 116]]]


ZeroDivisionError: division by zero